#### Notebook Purpose:

In this notebook, I plan to show you how I created our database schema and structure. In the first step, I highlight the statements I used to create the tables and relationships and attached a image of the schema. In the next step, I clean and seperate all of the data into different dataframes, so they can be loaded into the database. Finally in the last step, I loaded all of the dataframes into the tables.

##### I ran all of the CREATE statments below using the Postgres CLI.

#### Create the "vehicle_make" table:

#### Create the "dealer" table:

#### Create the "rate" table:

#### Create the "city" table:

#### Create the "vehicle" table:

#### Create the "vehicle_stats" table:

#### Create the "vehicle_posting" table:

#### Entity Relationship Diagram for all of these tables:

![erd](./images/ERD.png)

#### CatCode, Clean, and Normalize the Data before Inserting:

In this step, I am going to add cat codes as IDs for attributes that are foreign keys.


In [191]:
import pandas as pd
import numpy as np
import psycopg2
import getpass
import sqlite3

In [192]:
carscom = pd.read_pickle('/dsa/groups/casestudy2022su/team05/carscom_v03.pkl')

In [193]:
# Create ID for Vehicle_Make
carscom['Make_ID'] = carscom['Make'].astype('category').cat.codes

In [194]:
# Create ID for Dealer
carscom['Dealer_ID'] = carscom['Dealer_Name'].astype('category').cat.codes

In [195]:
# Create ID for Rate
carscom['Rate_ID'] = carscom['Rate'].astype('category').cat.codes

In [196]:
# Create ID for Vehicle
carscom['Vehicle_ID'] = carscom['Year'].map(str) + '-' + carscom['Make_ID'].map(str) + '-' + carscom['Model']
carscom['Vehicle_ID'] = carscom['Vehicle_ID'].astype('category').cat.codes

In [197]:
# Create ID for Vehicle_Posting
carscom['Posting_ID'] = range(1, len(carscom) + 1)

In this step, I am going to normalize, seperate, and clean the data into the individual dataframes. This will allow me to load the data into the database much more efficently.

In [198]:
# Create dataframe for Dealer table:
dealer_table = carscom[['Dealer_ID', 'Dealer_Name']]
dealer_table = dealer_table.drop_duplicates(inplace = False)
dealer_table.columns = ['Dealer_ID', 'Dealer']

In [199]:
# Create dataframe for Vehicle Make table:
vehicle_make_table = carscom[['Make_ID', 'Make']]
vehicle_make_table = vehicle_make_table.drop_duplicates(inplace = False)

In [200]:
# Create dataframe for Rate table:
rate_table = carscom[['Rate_ID', 'Rate']]
rate_table = rate_table.drop_duplicates(inplace = False)

In [201]:
# Create dataframe for City table:
city_table = carscom[['Zip', 'City', 'State', 'Distance Radius', 'Lon', 'Lat']]
city_table = city_table.drop_duplicates(inplace = False)
city_table.columns = ['Zip_Code', 'City', 'State', 'Distance_Radius', 'Longitude', 'Latitude']
city_table = city_table.drop([243768,243771])

In [202]:
# Create dataframe for Vehicle Stats table:
vehicle_stats_table = carscom[['Vehicle_ID', 'Avg_Price', 'Min_Price', 'Max_Price', 'Avg_Mileage', 'Min_Mileage', 'Max_Mileage', 'Count']]
vehicle_stats_table = vehicle_stats_table.drop_duplicates(inplace = False)

In [203]:
# Create dataframe for Vehicle table:
vehicle_table = carscom[['Vehicle_ID', 'Year', 'Make_ID', 'Model']]
vehicle_table = vehicle_table.drop_duplicates(inplace = False)

In [204]:
# Create dataframe for Vehicle Posting table:
vehicle_posting_table = carscom[['Posting_ID', 'Vehicle_ID', 'Dealer_ID', 'Rate_ID', 'Mileage', 'Price', 'Under_Value($)', 'miles', 'Zip', 'electronic_dealer']]
vehicle_posting_table = vehicle_posting_table.drop_duplicates(inplace = False)
vehicle_posting_table.columns = ['Posting_ID', 'Vehicle_ID', 'Dealer_ID', 'Rate_ID', 'Mileage', 'Price', 'Under_Value', 'Miles', 'Zip_Code', 'Electronic_Posting']

In this step, I am going to load the dataframes into the database using Python.

In [125]:
mypasswd = getpass.getpass()

connection = psycopg2.connect(database='casestdysu22t05', user='mtrkkf', host='pgsql.dsa.lan', password=mypasswd)

cursor = connection.cursor()

del mypasswd

········


In [143]:
# Create INSERT statement for Dealer table:
INSERT_SQL = 'INSERT INTO dealer '
INSERT_SQL += ' (dealer_id,dealer_name) VALUES '
INSERT_SQL += '(%s,%s)'
print(INSERT_SQL)

INSERT INTO dealer  (dealer_id,dealer_name) VALUES (%s,%s)


In [144]:
# Load dataframe into Dealer table:
with connection, connection.cursor() as cursor:
    for row in dealer_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)

In [145]:
# Create INSERT statement for Rate table:
INSERT_SQL = 'INSERT INTO rate '
INSERT_SQL += ' (rate_id,rate) VALUES '
INSERT_SQL += '(%s,%s)'
print(INSERT_SQL)

INSERT INTO rate  (rate_id,rate) VALUES (%s,%s)


In [146]:
# Load dataframe into Rate table:
with connection, connection.cursor() as cursor:
    for row in rate_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)

In [147]:
# Create INSERT statement for Vehicle Make table:
INSERT_SQL = 'INSERT INTO vehicle_make '
INSERT_SQL += ' (make_id,make) VALUES '
INSERT_SQL += '(%s,%s)'
print(INSERT_SQL)

INSERT INTO vehicle_make  (make_id,make) VALUES (%s,%s)


In [148]:
# Load dataframe into Vehicle Make table:
with connection, connection.cursor() as cursor:
    for row in vehicle_make_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)

In [152]:
# Create INSERT statement for City table:
INSERT_SQL = 'INSERT INTO city '
INSERT_SQL += ' (zip_code,city,state,distance_radius,longitude,latitude) VALUES '
INSERT_SQL += '(%s,%s,%s,%s,%s,%s)'
print(INSERT_SQL)

INSERT INTO city  (zip_code,city,state,distance_radius,longitude,latitude) VALUES (%s,%s,%s,%s,%s,%s)


In [206]:
# Load dataframe into City table:
with connection, connection.cursor() as cursor:
    for row in city_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)

In [207]:
# Create INSERT statement for Vehicle table:
INSERT_SQL = 'INSERT INTO vehicle '
INSERT_SQL += ' (Vehicle_ID,Year,Make_ID,Model) VALUES '
INSERT_SQL += '(%s,%s,%s,%s)'
print(INSERT_SQL)

INSERT INTO vehicle  (Vehicle_ID,Year,Make_ID,Model) VALUES (%s,%s,%s,%s)


In [208]:
# Load dataframe into Vehicle table:
with connection, connection.cursor() as cursor:
    for row in vehicle_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)

In [225]:
# Create INSERT statement for Vehicle Stats table:
INSERT_SQL = 'INSERT INTO vehicle_stats ' 
INSERT_SQL += ' (Vehicle_ID,Avg_Price,Min_Price,Max_Price,Avg_Mileage,Min_Mileage,Max_Mileage,Count) VALUES '
INSERT_SQL += '(%s,%s,%s,%s,%s,%s,%s,%s)'
print(INSERT_SQL)

INSERT INTO vehicle_stats  (Vehicle_ID,Avg_Price,Min_Price,Max_Price,Avg_Mileage,Min_Mileage,Max_Mileage,Count) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)


In [226]:
# Load dataframe into Vehicle Stats table:
with connection, connection.cursor() as cursor:
    for row in vehicle_stats_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)

In [228]:
# Create INSERT statement for Vehicle Posting table:
INSERT_SQL = 'INSERT INTO vehicle_posting ' 
INSERT_SQL += ' (Posting_ID,Vehicle_ID,Dealer_ID,Rate_ID,Mileage,Price,Under_Value,Miles,Zip_Code,Electronic_Posting) VALUES '
INSERT_SQL += '(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
print(INSERT_SQL)

INSERT INTO vehicle_posting  (Posting_ID,Vehicle_ID,Dealer_ID,Rate_ID,Mileage,Price,Under_Value,Miles,Zip_Code,Electronic_Posting) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)


In [229]:
# Load dataframe into Vehicle Posting table:
with connection, connection.cursor() as cursor:
    for row in vehicle_posting_table.itertuples(index = False, name = None):
        cursor.execute(INSERT_SQL, row)